In [ ]:
import os
import dlib
import glob

import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm

import numpy as np
%matplotlib inline

In [ ]:
img_black = np.zeros((70,140,3))

base_dir = 'data'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

classes = 'Begin, Choose, Connection, Navigation, Next, Previous, Start, Stop, Hello, Web'
classes = classes.split(', ')

predictor_path = 'assests/predictors/shape_predictor_68_face_landmarks.dat'

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)

train_people = 'F01, F02, F04, F05, F06, F07, F08, F09, M01, M02, M04'.split(', ')
val_people = 'F10, M07'.split(', ')
test_people = 'F11, M08'.split(', ')
print(train_people)
print(val_people)
print(test_people)

classes_num = ['0'+str(i) if i < 10 else str(i) for i in range(1, 11) ]
word_ids = ['0'+str(i) if i < 10 else str(i) for i in range(1, 11) ]
classes_dict = dict(zip(classes_num, classes))
print(classes_num)
print(word_ids)
print(classes_dict)

In [ ]:
def show_img(img, figsize=(8, 8)):
    fig, ax = plt.subplots(figsize=figsize)
    ax.grid(False)
    ax.set_yticklabels([])
    ax.set_xticklabels([])
    ax.imshow(img)
    plt.imshow(img)

In [ ]:
train_image = []
train_class = []

for class_id in tqdm(range(len(classes))):
    images = sorted(glob.glob('data/train/' + classes[class_id] + '/*.jpg'))
    for i in range(len(images)):
        if ('noised' in images[i] or 'rand_contr' in images[i]
        or 'vert_flip' in images[i] or 'hor_flip' in images[i]):
            train_image.append(images[i].split('/')[3])
            train_class.append(classes_dict[images[i].split('/')[3].split(']')[-1][:2]])
        else:
            train_image.append(images[i].split('/')[3])
            train_class.append(classes_dict[images[i].split('/')[3].split('_')[1]])
        
train_data = pd.DataFrame()
train_data['image'] = train_image
train_data['class'] = train_class

train_data.to_csv('data/miracl/train_new.csv',header=True, index=False)


val_image = []
val_class = []

for class_id in tqdm(range(len(classes))):
    images = sorted(glob.glob('data/validation/' + classes[class_id] + '/*.jpg'))
    for i in range(len(images)):
        val_image.append(images[i].split('/')[3])
        val_class.append(classes_dict[images[i].split('/')[3].split('_')[0]])
val_data = pd.DataFrame()
val_data['image'] = val_image
val_data['class'] = val_class

val_data.to_csv('data/miracl/val_new.csv',header=True, index=False)

In [ ]:
train = pd.read_csv('data/miracl/train_new.csv')
val = pd.read_csv('data/miracl/val_new.csv')

y_tr = train['class']
y_tr_dummy = pd.get_dummies(y_tr)

y_val = val['class']
y_val_dummy = pd.get_dummies(y_val)

In [ ]:
train.head()

In [ ]:
train.tail()

In [ ]:
val.head()

In [ ]:
val.tail()

In [ ]:
print(len(train))
print(len(val))

In [1]:
import tensorflow as tf
from tensorflow import keras

print(tf.__version__)
print(keras.__version__)

1.15.0
2.2.4-tf


In [2]:
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU')]

In [ ]:
from tweaked_ImageGenerator_v2 import ImageDataGenerator

In [ ]:
datagen = ImageDataGenerator()
train_data = datagen.flow_from_directory('data/train', target_size=(70, 140), batch_size=2, frames_per_step=22, shuffle=False)

In [ ]:
val_data = datagen.flow_from_directory('data/validation', target_size=(70, 140), batch_size=2, frames_per_step=22, shuffle=False)

In [ ]:
import math

print(math.ceil(8520/(2*22)))
print(math.ceil((1510/(2*22))))

In [ ]:
model_small = keras.models.Sequential()

model_small.add(keras.layers.ZeroPadding3D(padding=(1, 2, 2), input_shape=(22, 50, 100, 3)))
model_small.add(keras.layers.Conv3D(32, (3, 5, 5), strides=(1, 2, 2), activation='relu'))
model_small.add(keras.layers.MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
model_small.add(keras.layers.Dropout(.3))

model_small.add(keras.layers.ZeroPadding3D(padding=(1, 2, 2)))
model_small.add(keras.layers.Conv3D(64, (3, 5, 5), strides=(1, 1, 1), activation='relu'))
model_small.add(keras.layers.MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
model_small.add(keras.layers.Dropout(.3))

model_small.add(keras.layers.ZeroPadding3D(padding=(1, 1, 1)))
model_small.add(keras.layers.Conv3D(96, (3, 3, 3), strides=(1, 2, 2), activation='relu'))
model_small.add(keras.layers.MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
model_small.add(keras.layers.Dropout(.3))

model_small.add(keras.layers.TimeDistributed(keras.layers.Flatten()))

model_small.add(keras.layers.Bidirectional(keras.layers.LSTM(256, activation='relu', return_sequences=False)))

model_small.add(keras.layers.Dense(10, activation='softmax'))

In [ ]:
adam2 = keras.optimizers.Adam(lr=0.0001)
save_weights = keras.callbacks.ModelCheckpoint('weight.hdf5', save_best_only=True, monitor='val_loss', mode='min')
model_small.compile(loss='categorical_crossentropy',optimizer=adam2,metrics=['accuracy'])
model_small.summary()

In [ ]:
history = model_small.fit(train_data, epochs=20, steps_per_epoch=130, 
                              validation_data=val_data, validation_steps=23, 
                              callbacks=[save_weights], shuffle=True)